In [1]:
import cv2
import os
import math
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
import skvideo.io

In [2]:
ROOT_DATA_DIR = "../raw_data/Mov_Files"
OUT_DATA_DIR = "../raw_data/Out_Frames"
OUT_NP_ARRAY_PATH = "../raw_data/Out_Np_Arrays"
OUT_FEATURE_CSVs_PATH = "../raw_data/Out_Feature_CSVs"
DATA_SUB_DIRs = ["Train", "Test"]
CLS_LIST = ["Bad", "Good"]
EXTs_LIST = ["mov"]
INPUT_SHAPE = [256, 256, 3]

In [3]:
##### Create Train and Test Dataset CSVs or DataFrames
def get_data_dict(data_path, class_list, ext_list):
    data_dict = {"vid_paths":[], "vid_files":[], "labels":[]}
    for i, clss in enumerate(class_list):
        dir_path = os.path.join(data_path, clss)
        list_vids = [l for l in os.listdir(dir_path) if l.split(".")[-1] in ext_list]
        print(f"Found {len(list_vids)} Videos in {dir_path} Directory")
        data_dict["vid_files"].extend(list_vids)
        data_dict["labels"].extend([i for k in range(len(list_vids))])
        data_dict["vid_paths"].extend([dir_path for _ in range(len(list_vids))])
    
    total_videos = len(data_dict["vid_files"])
    total_labels = len(data_dict["labels"])
    print(f"Number of Videos : {total_videos}")
    print(f"Number of Labels : {total_labels}")
    return data_dict

train_data_dict = pd.DataFrame(get_data_dict(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[0]),
                               class_list=CLS_LIST,
                               ext_list=EXTs_LIST))


test_data_dict = pd.DataFrame(get_data_dict(data_path=os.path.join(ROOT_DATA_DIR, DATA_SUB_DIRs[1]),
                               class_list=CLS_LIST,
                               ext_list=EXTs_LIST))       

Found 40 Videos in ../raw_data/Mov_Files/Train/Bad Directory
Found 40 Videos in ../raw_data/Mov_Files/Train/Good Directory
Number of Videos : 80
Number of Labels : 80
Found 10 Videos in ../raw_data/Mov_Files/Test/Bad Directory
Found 10 Videos in ../raw_data/Mov_Files/Test/Good Directory
Number of Videos : 20
Number of Labels : 20


In [4]:
train_data_dict.sample(frac=1).head()

,vid_paths,vid_files,labels
46,../raw_data/Mov_Files/Train/Good,25.mov,1
35,../raw_data/Mov_Files/Train/Bad,77.mov,0
12,../raw_data/Mov_Files/Train/Bad,50.mov,0
75,../raw_data/Mov_Files/Train/Good,9.mov,1
16,../raw_data/Mov_Files/Train/Bad,69.mov,0


In [5]:
test_data_dict.sample(frac=1).head()

,vid_paths,vid_files,labels
3,../raw_data/Mov_Files/Test/Bad,99.mov,0
8,../raw_data/Mov_Files/Test/Bad,93.mov,0
12,../raw_data/Mov_Files/Test/Good,88.mov,1
5,../raw_data/Mov_Files/Test/Bad,92.mov,0
13,../raw_data/Mov_Files/Test/Good,89.mov,1


In [6]:
#### Function to convert Videos into Frames
def videos_to_frame(dataframe, out_data_path, class_list, max_frames):
    for index, row in dataframe.iterrows():
        video_to_convert = os.path.join(row["vid_paths"], row["vid_files"])
        out_frames_path = os.path.join(out_data_path, class_list[row["labels"]], row["vid_files"].split(".")[0])
        try:
            os.makedirs(out_frames_path, exist_ok=True)
        except:
            print("File Already Created")
        print(f"Converting {video_to_convert} and Storing Frames in {out_frames_path}")
        cap=skvideo.io.vread(video_to_convert)
        
        for frameId, frame in enumerate(cap):
            filename =f"frame-{str(int(frameId)).zfill(7)}.jpg"
            cv2.imwrite(os.path.join(out_frames_path, filename), cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        
# videos_to_frame(dataframe=train_data_dict,
#                 out_data_path=os.path.join(OUT_DATA_DIR, DATA_SUB_DIRs[0]),
#                 class_list=CLS_LIST,
#                 max_frames=100)

# videos_to_frame(dataframe=test_data_dict,
#                 out_data_path=os.path.join(OUT_DATA_DIR, DATA_SUB_DIRs[1]),
#                 class_list=CLS_LIST,
#                 max_frames=100)

In [16]:
resnet50 = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
resnet50.trainable = False
feature = resnet50.output
feature = tf.keras.layers.GlobalAveragePooling2D(name='feature')(feature)
model_arch = tf.keras.Model(inputs=resnet50.input, outputs=feature)
print(model_arch.summary())

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 128, 128, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [8]:
def get_feature_vector(model, input_frame):
    feature = model.predict(input_frame)
    return feature

def videos_to_features_npy(model, dataframe, out_data_path, class_list, max_frames):
    X = []
    for index, row in dataframe.iterrows():
        vid_features = []
        video_to_convert = os.path.join(row["vid_paths"], row["vid_files"])
        print(f"Converting {video_to_convert} and Storing Features")
        cap = skvideo.io.vread(video_to_convert)
        for frame in cap:
            preprocessed_frame = cv2.resize(frame, (INPUT_SHAPE[0], INPUT_SHAPE[1]))
            preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
            preprocessed_frame = tf.keras.applications.resnet.preprocess_input(preprocessed_frame)
            feature = list(get_feature_vector(model, preprocessed_frame)[0])
            
            vid_features.append(feature)
        X.append(vid_features)
    np.save(out_data_path, X)
        
# videos_to_features_npy(model=model_arch,
#                      dataframe=train_data_dict,
#                      out_data_path=os.path.join(OUT_NP_ARRAY_PATH, "train_features.npy"),
#                      class_list=CLS_LIST,
#                      max_frames=100)

# videos_to_features_npy(model=model_arch,
#                      dataframe=test_data_dict,
#                      out_data_path=os.path.join(OUT_NP_ARRAY_PATH, "test_features.npy"),
#                      class_list=CLS_LIST,
#                      max_frames=100)    
    

In [11]:
def get_feature_vector(model, input_frame):
    feature = model.predict(input_frame)
    return feature

def videos_to_features_csv(model, dataframe, out_data_path, class_list, max_frames):
    for index, row in dataframe.iterrows():
        csv_dict = {}
        feature_dim = model.output.shape[1]
        for i in range(feature_dim):
            csv_dict[f"feature_{str(i)}"] = []
        
        video_to_convert = os.path.join(row["vid_paths"], row["vid_files"])
        out_csv_path = os.path.join(out_data_path, class_list[row["labels"]])
        try:
            os.makedirs(out_csv_path, exist_ok=True)
        except:
            print("File Already Created")
        
        out_csv_file = os.path.join(out_csv_path, row["vid_files"].split(".")[0]+".csv")
        print(f"Converting {video_to_convert} and Storing Features in {out_csv_file} File")
        
        cap = skvideo.io.vread(video_to_convert)
        for frame in cap:
            preprocessed_frame = cv2.resize(frame, (INPUT_SHAPE[0], INPUT_SHAPE[1]))
            preprocessed_frame = np.expand_dims(preprocessed_frame, axis=0)
            preprocessed_frame = tf.keras.applications.resnet.preprocess_input(preprocessed_frame)
            feature = list(get_feature_vector(model, preprocessed_frame)[0])
            
            for i, val in enumerate(feature):
                csv_dict[f"feature_{str(i)}"].append(val)
        pd.DataFrame(csv_dict).to_csv(out_csv_file)
        
# videos_to_features_csv(model=model_arch,
#                      dataframe=train_data_dict,
#                      out_data_path=os.path.join(OUT_FEATURE_CSVs_PATH, DATA_SUB_DIRs[0]),
#                      class_list=CLS_LIST,
#                      max_frames=100)

# videos_to_features_csv(model=model_arch,
#                      dataframe=test_data_dict,
#                      out_data_path=os.path.join(OUT_FEATURE_CSVs_PATH, DATA_SUB_DIRs[1]),
#                      class_list=CLS_LIST,
#                      max_frames=100)   

Converting ../raw_data/Mov_Files/Train/Bad/74.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/74.csv File
Converting ../raw_data/Mov_Files/Train/Bad/53.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/53.csv File
Converting ../raw_data/Mov_Files/Train/Bad/70.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/70.csv File
Converting ../raw_data/Mov_Files/Train/Bad/62.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/62.csv File
Converting ../raw_data/Mov_Files/Train/Bad/65.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/65.csv File
Converting ../raw_data/Mov_Files/Train/Bad/44.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/44.csv File
Converting ../raw_data/Mov_Files/Train/Bad/56.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/56.csv File
Converting ../raw_data/Mov_Files/Train/Bad/71.mov and Storing Features in ../raw_data/Out_Feature_CSVs/Train/Bad/71.csv File


In [ ]:
from keras.layers import TimeDistributed, Conv2D, Dense, MaxPooling2D, Flatten, LSTM, Dropout, BatchNormalization
from keras import models
model_cnlst = models.Sequential()
model_cnlst.add(TimeDistributed(Conv2D(128, (3, 3), strides=(1,1),activation='relu'),input_shape=(10, 250, 250, 1)))
model_cnlst.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(MaxPooling2D(2,2)))
model_cnlst.add(TimeDistributed(Conv2D(64, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(Conv2D(32, (3, 3), strides=(1,1),activation='relu')))
model_cnlst.add(TimeDistributed(MaxPooling2D(2,2)))
model_cnlst.add(TimeDistributed(BatchNormalization()))


model_cnlst.add(TimeDistributed(Flatten()))
model_cnlst.add(Dropout(0.2))

model_cnlst.add(LSTM(32,return_sequences=False,dropout=0.2, activation='tanh')) # used 32 units
model_cnlst.add(Dense(64,activation='relu'))
model_cnlst.add(Dense(32,activation='relu'))
model_cnlst.add(Dropout(0.2))
model_cnlst.add(Dense(1, activation='sigmoid'))
model_cnlst.summary()

In [ ]:
import keras

In [ ]:
train_labels = pd.concat([train_set.iloc[:30,:][['label']],train_set.iloc[50:,:][['label']]])

In [ ]:
validation_labels = train_set.iloc[30:50,:]['label']

In [ ]:
from keras import optimizers
callbacks_list_cnlst=[keras.callbacks.EarlyStopping(
monitor='acc',patience=3),
               keras.callbacks.ModelCheckpoint(
               filepath='cnn_lstm_model_new3.h5',
               monitor='val_loss',
               save_best_only=True),
                keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.1, patience = 3)
               ]


optimizer_new=optimizers.RMSprop(lr=0.01)
model_cnlst.compile(optimizer=optimizer_new,loss='binary_crossentropy',metrics=['acc'])
# Training:
history_new_cnlst=model_cnlst.fit(train_dataset_new,train_labels,batch_size=10,epochs=20,
                        validation_data=(validation_dataset_new,validation_labels),
                       callbacks=callbacks_list_cnlst)

In [ ]:
np.mean(history_new_cnlst.history['acc'])